# Create maps of Myanmar with reservoirs and reservoir emissions
### T. JANUS, email: tomasz.k.janus@gmail.com , tomasz.janus@manchester.ac.uk
#### Last modified: 27-June-2024

## ALSO:
### 1. Processes discounted emissions from multipurpose reservoirs by taking into account emissions associated with agricultural use
### 2. Incorporates emissions of RoR hydropower by introducing a common emission intensity per generated energy
### 3. Creates a combined dataframe for visualisation purposes and as a source of information for subsequent optimal dam selection.



### Load libraries and data

In [ ]:
# Load libraries
library(tidyverse)
library(dplyr)
library(sf)
library(Hmisc)
library(ggplot2)
library(colorspace)
library(ggspatial)
library(latex2exp)
library(readxl)
library(ggnewscale)
library(writexl) 
library(ggthemes)
library(ggExtra)
library(ggbreak)

mya_epsg_code <- "EPSG:32646"
googlecrs <- "EPSG:4326"

text_fontsize <- 12
axis_label_fontsize <- 13
legend_text_fontsize <- 10
legend_title_fontsize <- 12
plot_title_fontsize <- 15
strip_fontsize <- 13

# Assumed emission intensity (per generated energy) of RoR hydroelectric plants
em_intensity <- 3 # gCO2e/kWh

## UNITS
1. Net emission fluxes (aka. net aerial emissions) in gCO$_{2e}$/m$^2$/year
2. Emission intensities of HP in gCO$_{2e}$/kWh, e.g. 3~gCO$_{2e}$/kWh
3. Total (annual) emissions in ktCO$_{2e}$year

## VARIABLES
* `co2_net`, `ch4_net`, `n2o_mean` are aerial emissions (fluxes) in gCO$_{2e}$/m$^2$/yr
* `tot_em_int` represents fluxes (misnamed intensity) in the same unit as `co2_net` etc.
* `tot_em` are total emissions in ktCO$_{2e}$/year
* `em_intensity` is the emission intensity of hydropower in (gCO$_{2e}$/kWh) - same as tCO$_{2e}$/GWh
* `ann_gen` is annual hydropower generation in (GWh/yr)
* `mean` is the mean hydropower generation in (MW)

## CONVERSIONS
- `tot_em` (ktCO$_{2e}$/yr)     <- `em_intensity` (gCO$_{2e}$kWh) * `ann_gen` (GWh/yr) / 1000
- `ann_gen` (GWh/yr)            <- `mean` (MW) * 24 * 365.25 / 1000

## 1. PROCESS RESERVOIRS ONLY (no RoRs)

In [ ]:
# Load the Myanmar country outline shape file and create a box (margins) around it
mya_outline <- sf::st_read(
    dsn = file.path("bin","gis_layers","myanmar_outline","Myanmar_outline.shp"), 
    quiet = TRUE, geometry_column = 'geometry') %>% 
    st_transform(mya_epsg_code)

df_sf <- sf::st_as_sf(mya_outline, coords=c("Lon", "Lat"), crs=googlecrs, agr = "identity")
df_xy <- sf::st_transform(df_sf, crs=mya_epsg_code)
bbox <- sf::st_bbox(df_xy)
# Expand box by 20% to give a little extra room
Dx <- (bbox[["xmax"]]-bbox[["xmin"]])*0.1
Dy <- (bbox[["ymax"]]-bbox[["ymin"]])*0.1
bbox["xmin"] <- bbox["xmin"] - Dx
bbox["xmax"] <- bbox["xmax"] + Dx
bbox["ymin"] <- bbox["ymin"] - Dy
bbox["ymax"] <- bbox["ymax"] + Dy
# 
bb <- c(bbox["xmin"], bbox["ymin"], bbox["xmax"], bbox["ymax"])

# Load and filter the shape data with reservoirs, rivers and neighbouring countries
mya_reservoirs <- sf::st_read(
    dsn = file.path("bin", "heet_outputs_MIN_LOW_PRI","reservoirs_updated.shp"), quiet = TRUE) %>% 
    st_transform(mya_epsg_code) %>% 
    filter(!is.na(tot_em)) %>% 
    mutate(type = replace(type, type == "unknown", "hydroelectric")) %>%
    rename(reservoir_type = type) %>%
    # Get some names in mya_reservoirs to the IFC format (some names are off as they were not 
    # coded properly in HEET)
    mutate(name = case_when(
        name == "Paung Laung (lower)" ~ "Lower Paunglaung", 
        name == "Paung Laung (upper)" ~ "Upper Paunglaung",
        name == "Thapanzeik" ~ "Thaphanseik",
        TRUE ~ name
    ))
# Arange rows in res_centroids (reemission output data with geometry) from highest to lowest area
res_centroids <- sf::st_centroid(mya_reservoirs, of_largest_polygon = TRUE) %>% arrange(desc(r_area_km2))
# Load shape files with rivers and neigbouring countries
mya_rivers <- sf::st_read(dsn = file.path("bin","gis_layers","mya_rivers.shp"), quiet = TRUE)  %>% 
    st_transform(mya_epsg_code)
mya_neighbours <- sf::st_read(
    dsn = file.path("bin","gis_layers","World_Countries_Generalized.shp"), quiet = TRUE)  %>% 
    st_transform(mya_epsg_code) %>%
    filter(COUNTRY != "Myanmar")
formatted_string <- paste(
    "Getting a cropped map with ",
    nrow(res_centroids), 
    "hydroelectric, multipurpose and irrigation reservoris.")
print(formatted_string)
mya_neighbours_crop <- st_crop(mya_neighbours, y = bbox)
num_hydroelectric = nrow(res_centroids %>% filter(reservoir_type == "hydroelectric"))
num_multipurpose = nrow(res_centroids %>% filter(reservoir_type == "multipurpose"))
num_irrigation = nrow(res_centroids %>% filter(reservoir_type == "irrigation"))
formatted_string <- paste(
    "Number of hydroelectric reservoirs: ",
    num_hydroelectric, 
    ". Number of multipurpose reservoirs: ",
    num_multipurpose,
    ". Number of irrigation reservoirs: ",
    num_irrigation)
print(formatted_string)
# Calculate total emissions (in thousands of tonnes CO2e/year)
res_centroids$tot_em_int = res_centroids$tot_em * res_centroids$r_area_km2 / 1000
print("Reservoir types:")
print(unique(res_centroids$reservoir_type))

In [ ]:
Hmisc::describe(res_centroids$tot_em_int)

In [ ]:
head(res_centroids,2)

# 2. CREATE A FACET MAP OF EMISSION INTENSITIES WITH ALL THREE RESERVOIR TYPES

#### Hints
* to remove basemap layer from the map, comment out `ggspatial::annotation_map_tile(
        type = "osm", zoom = 6, alpha=0.4, interpolate = TRUE, quiet = TRUE) +`
* increase alpha in `geom_sf(data=mya_neighbours_crop, fill="lightgrey", col="black", alpha=0.1) +` from 0.1 to 0.5
* increase alpha in `geom_sf(data=mya_outline, fill="antiquewhite1", col="black", alpha=0.8, size = 5.5)` to 0.7 to 0.8

## 2A. Maps with net (per area) GHG emissions for all three types of reservoirs - hydroelectric, multipurpose and irrigation

In [ ]:
# Bin net (per-surface-area) emissions
em_ranges <- c(
    "< 500", 
    "500 - 1000", 
    "1000 - 2000", 
    "2000 - 3000",
    "3000 - 5000",
    "> 5000"
)
res_centroids <- 
    res_centroids %>%
    mutate(
        em_range = factor(case_when(
            `tot_em` < 500 ~ em_ranges[1],
            between(`tot_em`, 500, 1000) ~ em_ranges[2],
            between(`tot_em`, 1000, 2000) ~ em_ranges[3],
            between(`tot_em`, 2000, 3000) ~ em_ranges[4],
            between(`tot_em`, 3000, 5000) ~ em_ranges[5],
            `tot_em` >= 5000 ~ em_ranges[6]
        ), levels = em_ranges)
    )
colors_map <- colorspace::diverging_hcl(6, palette = "Tropic", rev = F) # OK: Cork
# Plot with ggplot2
mya_gg <- ggplot(lims_method = "geometry_bbox")
mya_gg +
    geom_sf(fill = "lightgrey", alpha=0.9) +
    geom_sf(data=mya_neighbours_crop, fill="lightgrey", col="black", alpha=0.5) +
    geom_sf(data=mya_outline, fill="antiquewhite1", col="black", alpha=0.8, size = 5.5) +
    geom_sf_label(data=mya_neighbours_crop, aes(label = COUNTRY), fill = "white") + 
    geom_sf(data=mya_rivers, col="lightblue3") +
    geom_sf(data = res_centroids %>% arrange(desc(r_area_km2)),
          pch = 21,
          aes(size = r_area_km2, fill = em_range),
          col = "Gray60") +
    scale_size(breaks = c(10, 50, 100, 300, 500),
        range = c(1, 15),
        guide = guide_legend(
          direction = "horizontal",
          ncol = 5)) +
    scale_fill_manual(
        values = colors_map, 
        name=TeX(
            r"(\overset{\normalsize{Net aerial emission}}{\overset{\normalsize{gCO$_{2e}$/m$^2$/yr}}})")) +
    labs(#title = "GHG emission intensities per reservoir type",
           sub = "FutureDAMS Project",
           #caption = "T. Janus, 2023",
           x = NULL, y = NULL,
           size = TeX(r"(\overset{\normalsize{Reservoir Area}}{\overset{\normalsize{km$^2$}}})")) +
    xlab(NULL) + ylab(NULL) +
    annotate(geom = "text", x = 820000, y = 1500000, label = "Andaman\nSea", 
     fontface = "italic", color = "grey22", size = 4) +
    annotate(geom = "text", x = 505000, y = 1950000, label = "Bay\nof\nBengal", 
     fontface = "italic", color = "grey22", size = 4) +
    ggspatial::annotation_scale(
        location = "tl",
        bar_cols = c("grey60", "white"),
        text_family = "Avenir Next Condensed") +
    ggspatial::annotation_north_arrow(
        location = "tl", which_north = "true",
        pad_x = unit(0.12, "in"), pad_y = unit(0.3, "in"),
        style = ggspatial::north_arrow_fancy_orienteering(
          fill = c("grey40", "white"),
          line_col = "grey20",
          text_family = "Avenir Next Condensed"))+
    facet_wrap(~reservoir_type) +
    xlim(bbox["xmin"]+Dx, bbox["xmax"]-Dx) + ylim(bbox["ymin"]+Dy, bbox["ymax"]-Dy) +
    coord_sf(crs = mya_epsg_code) +
    theme_minimal() +
    theme(
        text = element_text("Avenir Next Condensed", size=text_fontsize),
        legend.text = element_text(size = legend_text_fontsize),
        legend.title = element_text(size=legend_title_fontsize),
        axis.text.x = element_text(size = axis_label_fontsize),
        axis.text.y = element_text(size = axis_label_fontsize),
        strip.text = element_text(face="bold", hjust = 0, size=strip_fontsize),
        plot.title = element_text(hjust = 0, size=plot_title_fontsize),
        plot.caption = element_text(face = "italic"),
        panel.grid.major = element_line(
            colour = gray(0.5), linetype = "dashed", size = 0.5), 
        panel.background = element_rect(fill = "aliceblue"), 
        panel.border = element_rect(fill = NA),
        panel.spacing = unit(1, "lines"),
        legend.direction = "horizontal", legend.position = "bottom",
        panel.ontop = FALSE
    )
ggsave(
    file.path("figures","maps","emission_intensities_per_res_type.svg"), 
    width = 27, height = 20, units = "cm")
ggsave(
    file.path("figures","maps","emission_intensities_per_res_type.png"), 
    width = 27, height = 20, units = "cm")
#ggsave("map-us-ggdraw.pdf", width = 5, height = 8) # Save to a pdf
dev.off()

## 2B. Maps with total emissions in ktCO$_{2e}$/yr for all three types of reservoirs - hydroelectric, multipurpose and irrigation - RoRs not included

In [ ]:
# Plot histogram of total emissions of reservoirs (includes hydroelectric, multipurpose and irrigation reservoirs)
# tot_em_int are in ktCO2e/year
hist(res_centroids$tot_em_int, breaks=25)

In [ ]:
# Create a categorical total emission variable by binning total emissions (in thousands of tonnes CO2e/yr)
# into ranges of values defined in the vector 'tot_em_ranges'
tot_em_ranges <- c(
    "< 3", 
    "3 - 30",  
    "30 - 50",
    "50 - 200",
    "200 - 400",
    "> 400"
)

res_centroids <- 
    res_centroids %>%
    mutate(
        tot_em_range = factor(case_when(
            `tot_em_int` < 3 ~ tot_em_ranges[1],
            between(`tot_em_int`, 3, 30) ~ tot_em_ranges[2],
            between(`tot_em_int`, 30, 50) ~ tot_em_ranges[3],
            between(`tot_em_int`, 50, 200) ~ tot_em_ranges[4],
            between(`tot_em_int`, 200, 400) ~ tot_em_ranges[5],
            `tot_em_int` >= 400 ~ tot_em_ranges[6]
        ), levels = tot_em_ranges)
    )

colors_map <- colorspace::diverging_hcl(6, palette = "Tropic", rev = F) # OK: Cork

mya_gg <- ggplot(lims_method = "geometry_bbox")
mya_gg +
    geom_sf(fill = "lightgrey", alpha=0.9) +
    geom_sf(data=mya_neighbours_crop, fill="lightgrey", col="black", alpha=0.5) +
    geom_sf(data=mya_outline, fill="antiquewhite1", col="black", alpha=0.8, size = 5.5) +
    geom_sf_label(data=mya_neighbours_crop, aes(label = COUNTRY), fill = "white") + 
    geom_sf(data=mya_rivers, col="lightblue3") +
    geom_sf(data = res_centroids %>% arrange(desc(r_area_km2)),
          pch = 21,
          aes(size = r_area_km2, fill = tot_em_range),
          col = "Gray60") +
    scale_size(breaks = c(10, 50, 100, 300, 500),
        range = c(1, 15),
        guide = guide_legend(
          direction = "vertical",
          nrow = 1)) +
    scale_fill_manual(
        values = colors_map, 
        guide = guide_legend(
            direction = "vertical",
            nrow = 3),
        name=TeX(r"(\overset{\normalsize{Total emission}}{\overset{\normalsize{ktCO$_{2e}$/yr}}})")) +
    labs(
        #title = "GHG emissions per reservoir type",
        sub = "FutureDAMS Project",
        #caption = "T. Janus, 2023",
        size = TeX(r"(\overset{\normalsize{Reservoir Area}}{\overset{\normalsize{km$^2$}}})")) +
    xlab(NULL) + ylab(NULL) +
    theme_minimal() +
    theme(
        text = element_text("Avenir Next Condensed", size=text_fontsize),
        legend.text = element_text(size = legend_text_fontsize),
        legend.title = element_text(size=legend_title_fontsize),
        legend.key.size = unit(1,"line"),
        axis.text.x = element_text(size = axis_label_fontsize),
        axis.text.y = element_text(size = axis_label_fontsize),
        strip.text = element_text(face="bold", hjust = 0, size=strip_fontsize),
        plot.title = element_text(hjust = 0, size=plot_title_fontsize),
        plot.caption = element_text(face = "italic"),
        panel.grid.major = element_line(
            colour = gray(0.5), linetype = "dashed", size = 0.5), 
        panel.background = element_rect(fill = "aliceblue"), 
        panel.border = element_rect(fill = NA),
        legend.direction = "hozirontal", legend.position = "bottom",
        panel.spacing = unit(0.7, "lines"),
        panel.ontop = FALSE
    ) +
    annotate(geom = "text", x = 820000, y = 1500000, label = "Andaman\nSea", 
     fontface = "italic", color = "grey22", size = 4) +
    annotate(geom = "text", x = 505000, y = 1950000, label = "Bay\nof\nBengal", 
     fontface = "italic", color = "grey22", size = 4) +
    ggspatial::annotation_scale(
        location = "tl",
        bar_cols = c("grey60", "white"),
        text_family = "ArcherPro Book") +
    ggspatial::annotation_north_arrow(
        location = "tl", which_north = "true",
        pad_x = unit(0.12, "in"), pad_y = unit(0.3, "in"),
        style = ggspatial::north_arrow_fancy_orienteering(
          fill = c("grey40", "white"),
          line_col = "grey20",
          text_family = "ArcherPro Book"))+
    facet_wrap(~reservoir_type) +
    xlim(bbox["xmin"]+Dx, bbox["xmax"]-Dx) + ylim(bbox["ymin"]+Dy, bbox["ymax"]-Dy) +
    coord_sf(crs = mya_epsg_code)
ggsave(
    file.path("figures","maps", "total_emissions_per_res_type.svg"), width = 27, height = 20, units = "cm")
ggsave(
    file.path("figures","maps", "total_emissions_per_res_type.png"), width = 27, height = 20, units = "cm")
#ggsave("map-us-ggdraw.pdf", width = 5, height = 8)
dev.off()

## 2C. Plot a minimal map for visual abstract

In [ ]:
colors_map <- colorspace::diverging_hcl(6, palette = "Tropic", rev = F) # OK: Cork

mya_gg <- ggplot(lims_method = "geometry_bbox")
mya_gg +
    geom_sf(fill = "lightgrey", alpha=0.9) +
    geom_sf(data=mya_outline, fill="antiquewhite1", col="black", alpha=1, size = 5.5) +
    geom_sf(data=mya_rivers, col="lightblue3") +
    geom_sf(data = res_centroids,
          pch = 24,
          size=2.5,
          alpha = 0.7,
          aes(fill = tot_em_range),
          col = "black") +
    scale_fill_manual(
        values = colors_map, 
        name=TeX(r"(\overset{\normalsize{Total emission}}{\overset{\normalsize{ktCO$_{2e}$/yr}}})")) +
    labs(#caption = "T. Janus, 2023",
           size = "Reservoir Area\nkm2") +
    xlab(NULL) + ylab(NULL) +
    theme_void() +
    theme(
        text = element_text("Avenir Next Condensed"),
        strip.text = element_text(face="bold", hjust = 0),
        plot.title = element_text(hjust = 0),
        plot.caption = element_text(face = "italic"),
        panel.background = element_blank(),
        panel.border = element_blank(),
        legend.direction = "horizontal", legend.position = "none",
        panel.ontop = FALSE
    ) +
    xlim(bbox["xmin"]+Dx, bbox["xmax"]-Dx) + ylim(bbox["ymin"]+Dy, bbox["ymax"]-Dy) +
    coord_sf(crs = mya_epsg_code)
ggsave(
    file.path("figures","maps","total_emissions_vis_abstract.png"), width = 27, height = 20, units = "cm")
ggsave(
    file.path("figures","maps","total_emissions_vis_abstract.svg"), width = 27, height = 20, units = "cm")
dev.off()

## 2D. Calculate total emissions by status (existing and future) including RoR hydroelectric
### NOTE (reminder): 
**tot_em_int** is a total emission of a reservoir in ktCO$_{2e}$/yr

**tot_em** is a unit per area emission in gCO$_{2e}$/m$^2$/yr

In [ ]:
### Import ifc database that contains information about statuses of hydroelectric and multipurpose dams
# Why are we filtering out Bult dams?
# Change statuses to Built and Future
ifc_dams <- sf::st_read(
    dsn = file.path("bin", "gis_layers", "ifc_database", "all_dams_replaced_refactored.shp"), 
    quiet = TRUE)  %>% 
    st_transform(mya_epsg_code) %>%
    filter(Status != "Built") %>%
    st_drop_geometry() %>%
    select("IFC_ID", "Status", "DAM_NAME") %>%
    mutate(Status = ifelse(
        Status %in% c(
            'Construction', 'MOU', 'LocMoU', 'Identified', 'GOM Plan', 'JVA', 'MOA',
            'Suspended', 'Covenant'), 'Future', Status))
# Set status in res_centroids to Existing by default.
res_centroids$Status = "Existing"

# Join the centroids dataframe with ifc_dams and replace the status to Future for Future IFC dams
res_centroids_joined = dplyr::left_join(res_centroids, ifc_dams, by = join_by(id == IFC_ID)) %>%
  st_transform(mya_epsg_code) %>%
  mutate(Status = ifelse(is.na(Status.y), Status.x, Status.y)) %>%
  select(-Status.x, -Status.y) %>%
  mutate(Status = ifelse(
      Status == "Future", "Future Hydroelectric", Status)) %>%
  mutate(Status = ifelse(
      Status == "Existing", "Existing Hydroelectric", Status)) %>%
  arrange(desc(r_area_km2))

# Find a list of future RoR hydroelectric from the IFC database
# Future ROR
ifc_dams_ror_future <- sf::st_read(
    dsn = file.path("bin","gis_layers","ifc_database","all_dams_replaced_refactored.shp"), 
    quiet = TRUE) %>%
    st_transform(mya_epsg_code) %>%
    filter(Status != "Built") %>%
    mutate(`RoR.or.Sto` = tolower(`RoR.or.Sto`)) %>%
    filter(`RoR.or.Sto` == 'ror') %>%
    mutate(Status = "Future Hydroelectric")
# Existing ROR
ifc_dams_ror_existing <- sf::st_read(
    dsn = file.path("bin","gis_layers","ifc_database","all_dams_replaced_refactored.shp"), 
    quiet = TRUE) %>%
    st_transform(mya_epsg_code) %>%
    filter(Status == "Built") %>%
    mutate(`RoR.or.Sto` = tolower(`RoR.or.Sto`)) %>%
    filter(`RoR.or.Sto` == 'ror') %>%
    mutate(Status = "Existing Hydroelectric")

ifc_dams_ror_all <- rbind(ifc_dams_ror_future, ifc_dams_ror_existing) 

# Find if any future RoRs are already included in the reservoirs list for GHG emission estimation 
# (some RoRs were misclassified as RoR whereas in fact they're small storage dams)
doubled_names_all <- dplyr::inner_join(res_centroids_joined %>% 
    st_drop_geometry(), ifc_dams_ror_all, by=join_by(name == DAM_NAME)) %>%
    select(name) %>%
    as.list()

doubled_names_future <- dplyr::inner_join(res_centroids_joined %>% 
    st_drop_geometry(), ifc_dams_ror_future, by=join_by(name == DAM_NAME)) %>%
    select(name) %>%
    as.list()

print("Doubled names of future dams")
print(doubled_names_future)

ifc_dams_ror_all_filt <- ifc_dams_ror_all %>% filter(!(DAM_NAME %in% doubled_names_all$name))
ifc_dams_ror_future_filt <- ifc_dams_ror_future %>% filter(!(DAM_NAME %in% doubled_names_future$name))

# Units in the IFC data
# Inst_cap - MW
# Annual.Gen - GWh
# Des_Head - m
head(ifc_dams_ror_all_filt %>% select('DAM_NAME', 'Inst_cap', 'Annual.Gen', 'Des_Head'))

### `ifc_dams_ror_all_filt` is a dataframe of RoR hydroelectric unit with removed doubled entries

In [ ]:
colnames(res_centroids_joined)

In [ ]:
res_centroids_joined %>% filter(name=="Thaphanseik")

# 3. Calculate splits between emissions allocated to hydroelectric and to agricultural production in multipurpose reservoirs
### ** The split usage is based on the amount of water going to turbine vs irrigation supply **
- **TODO:** Check where the irrigation water should be abstracted from - Sedawgyi or Sedawgyi (upper)
- **NOTE:** There are no MultiPurpose reservoirs in the Salween river basin

In [ ]:
### Calculate flow proportions HP / Irrigation splits
sittaung_averages <- read_excel(
    file.path("outputs/pywr_sim_aligned_models/outputs_sittaung_hist_with_flow_rec.xlsx")) %>%
    slice(-c(1:2)) %>%       # Remove first two rows
    select((ncol(.)-9):ncol(.)-1) %>% # Select last six columns
    summarise_all(mean, na.rm = TRUE)
pair_indices <- seq(1, ncol(sittaung_averages), by = 2)
turbine_flows <- colSums(sittaung_averages[, pair_indices], na.rm = TRUE)
irr_flows <- colSums(sittaung_averages[, pair_indices + 1], na.rm = TRUE)
sittaung_hp_ratios <- turbine_flows / (irr_flows + turbine_flows)
irrawaddy_averages <- read_excel(
    file.path("outputs/pywr_sim_aligned_models/outputs_irrawaddy_hist_with_flow_rec.xlsx")) %>%
    slice(-c(1:2)) %>%       # Remove first two rows
    select((ncol(.)-11):ncol(.)) %>% # Select last sixteen columns
    summarise_all(mean, na.rm = TRUE)
pair_indices <- seq(1, ncol(irrawaddy_averages), by = 2)
turbine_flows <- colSums(irrawaddy_averages[, pair_indices], na.rm = TRUE)
irr_flows <- colSums(irrawaddy_averages[, pair_indices + 1], na.rm = TRUE)
irrawaddy_hp_ratios <- turbine_flows / (irr_flows + turbine_flows)

df1 <- data.frame(sittaung_hp_ratios) %>% rename(hp_ratio = sittaung_hp_ratios)
df2 <- data.frame(irrawaddy_hp_ratios) %>% rename(hp_ratio = irrawaddy_hp_ratios)
hp_ratios <- rbind(df1, df2)
new_row_names <- sub("_Turbine_Flow", "", rownames(hp_ratios))
rownames(hp_ratios) <- new_row_names
hp_ratios$irr_ratio = 1 - hp_ratios$hp_ratio
hp_ratios$name <- rownames(hp_ratios)

print("HP/total flow proportions for  multipurpose reservoirs")
hp_ratios

In [ ]:
### Fill the proportions manually and update the multipurpose reservoir table
# The following three vectors are filled in manually from the values calculated in the previous step.
# This step can be automated but due to time-constraints, we speed up by putting in the numbers by hand.
reservoir_names <- c(
    "Kinda", "Myittha", "Myogyi", "Zawgyi II", "Sedawgyi", "Thaphanseik",
    "Buywa (upper)", "Mone Chaung", "Kyee Ohn Kyee Wa", "Buywa", "Sedawgyi (upper)",
    "Lower Paunglaung", "Kabaung", "Phyu Chaung","Kun Chaung", "Yenwe"
    )
river_basin <- c(
    "irrawaddy", "irrawaddy", "irrawaddy", "irrawaddy", "irrawaddy", "irrawaddy", 
    "irrawaddy", "irrawaddy", "irrawaddy", "irrawaddy", "irrawaddy",
    "sittaung", "sittaung", "sittaung", "sittaung", "sittaung"
    )
# Values calculated manually from the outputs (excel files from pywer simulations)
percent_hp <- c(
    0.67, 0.94, 0.83, 0.35, 0.76, 0.53,
    1.0, 1.0, 1.0, 1.0, 1.0,
    0.94, 0.69, 0.74, 0.82, 0.83)

### Create a new dataframe `total_emissions_multi_merged` with emissions of multipurpose reservoirs

In [ ]:
# Mone Chaung, Buywa, Buywa (upper) and Kyee Ohn Kyee Wa store water for mazali irrigation. 
# They have volumes of 0.61, 0.99, 4.25, and 0.21 (6.06 km3 in total)
# let's water abstraction for irrigation based on relative volumes of each reservoirs and taking
# irrigation water consumption from Mazali irrigation abstraction
# CORRECTION: the irrigation flow is 5% of total flow. Therefore we have not used 
# The fractions computed are:
# --------------------------------------------
# vol_fractions <- c(0.61, 0.99, 4.25, 0.21)
# vol_fractions / sum(vol_fractions)
# --------------------------------------------
# Result: 0.1, 0.163, 0.70, 0.035
hp_fractions_multi <- data.frame(name = reservoir_names, basin = river_basin, hp_fraction = percent_hp)
# Get the data from the IFC database that only lists multipurpose reservoirs
hp_fractions_multi <- data.frame(name = reservoir_names, basin = river_basin, hp_fraction = percent_hp)
# Get the data from the IFC database that only lists multipurpose reservoirs
total_emissions_multi <- res_centroids_joined %>%
  filter(!(DAM_NAME %in% doubled_names_all$name)) %>%
  filter(reservoir_type == "multipurpose") %>%
  select(name, Status, reservoir_type, r_volume_m, r_area_km2, r_mean_dep, tot_em, tot_em_int) %>%
  mutate(volume_km3 = r_volume_m / 10^9) %>%
  select(!r_volume_m)
# Find emission splits between hp and irrigation for multipurpose reservoirs
total_emissions_multi_merged <- merge(total_emissions_multi, hp_fractions_multi, by="name") %>%
    mutate(tot_em_hp=tot_em_int*hp_fraction, tot_em_irr=tot_em_int*(1-hp_fraction)) %>%
    select(
        name, Status, reservoir_type, r_area_km2, r_mean_dep, tot_em, volume_km3, 
        hp_fraction, tot_em_int, tot_em_hp, tot_em_irr)

In [ ]:
head(total_emissions_multi_merged, 3)

In [ ]:
multi_summary <- total_emissions_multi_merged %>%
    group_by(Status) %>%
    summarise(
        emissions_total_hp = sum(tot_em_hp, na.rm = TRUE),
        emissions_total_irr = sum(tot_em_irr, na.rm = TRUE),
        emissions_total = sum(tot_em_int, na.rm = TRUE),
        total_area = sum(r_area_km2, na.rm = TRUE),
        total_volume = sum(volume_km3, na.rm = TRUE),
        mean_depth = mean(r_mean_dep, na.rm = TRUE),
        num = n()
  ) 
# total_emissions_multi_merged provides unit net emissions of MULTIPURPOSE reservoirs 
# (per surface area) divided between hydroelectric and agricultural uses
multi_summary

In [ ]:
### Find emission statistics of MULTIPURPOSE RESERVOIRS
emissions_multi_by_utility <- multi_summary %>%
  select(-emissions_total) %>%
  pivot_longer(cols = starts_with("emissions_total"),
               names_to = "utility",
               values_to = "emissions_total") %>%
  mutate(utility = ifelse(utility == "emissions_total_hp", "hp", "irr")) %>%
  group_by(Status, utility) %>%
  summarise(
      n = num,
      total_area = sum(total_area, na.rm = TRUE),
      total_volume = sum(total_volume, na.rm = TRUE),
      mean_depth = mean(mean_depth, na.rm = TRUE),
      total_emission = sum(emissions_total, na.rm = TRUE)
      )
emissions_multi_by_utility

<b style='color:red;'>NOTE</b> the dataframe with discounted emissions of multipurpose reservoirs, i.e. `total_emissions_multi_merged` **has not been saved** <b style='color:red;'>!!!!!!!!!</b>

# 4. Calculate total emissions and emisssions per GWh - include RoR and plot as triangles

## 4A. Calculate emissions of RoR plants using a constant emission intensity - see `em_intensity` variable and merged the calculated emissions and emission intensities into a dataframe with reservoir-based hydroelectric dams

In [ ]:
# Get emissions of RoR plants from pywr output excel file(s)
ror_em_intensity <- em_intensity

# Merge HP production figures into res_centroids data
emissions_data <- readxl::read_excel(
    file.path("intermediate", "merged_table.xlsx")) %>%
    distinct(ifc_id, .keep_all = TRUE) %>%
    select(
        ifc_id, dam_name, status_2_ifc, ro_r_or_sto_ifc, hp_type_reem, 
        res_area, mean, pctile_2, pctile_3, pctile_5, pctile_10) %>%
    rename(name=dam_name) %>%
    mutate(ann_gen = mean * 24 * 365.25 / 1000)
    # Annual generation in GWh.year

# Geospatial data from heet and reemission
hp_dams <- res_centroids_joined %>%
    filter(reservoir_type!='irrigation')

# Merge the geospatial dataframe from heet and reemission with hp production estimates from 
# pywr (emissions data)
# Represents only the delineated hydroelectric reservoirs, not ror plants
joined_data <- merge(hp_dams, emissions_data, by = "name", all.x=TRUE) %>%
    mutate(em_intensity = tot_em_int / ann_gen * 1000) %>%
    mutate(
        Status = ifelse(Status == 'Existing Reservoirs', 'Existing Hydroelectric', Status))
#Hmisc::describe(joined_data$em_intensity)

# Find HP generation for RoR plants from emissions_data from pywr filtered by hp type (sto/ror)
emissions_ror <- emissions_data %>%
    filter(hp_type_reem == 'ror') %>%
    mutate(emission_tot_int = ann_gen * ror_em_intensity / 1000)

# Create a dataframe of RoR hydropower with emissions and emission intensity information
# x [GWh/year] * y [gCO2/kWh] * 1,000,000 kWh / GWh / 1,000,000 gCO2/tonneCO2 -> X[tCO2/year]
# Divide by 1000 to get emission in ktCO2/year
ror_with_emissions_gis <- 
    merge(x=ifc_dams_ror_all_filt, y=emissions_ror, by.x="IFC_ID", by.y="ifc_id") %>%
    mutate(
        tot_em_range = factor(case_when(
            `emission_tot_int` < 3 ~ tot_em_ranges[1],
            between(`emission_tot_int`, 3, 30) ~ tot_em_ranges[2],
            between(`emission_tot_int`, 30, 50) ~ tot_em_ranges[3],
            between(`emission_tot_int`, 50, 200) ~ tot_em_ranges[4],
            between(`emission_tot_int`, 200, 400) ~ tot_em_ranges[5],
            `emission_tot_int` >= 400 ~ tot_em_ranges[6]
        ), levels = tot_em_ranges)
    )

In [ ]:
head(ror_with_emissions_gis,2)

## 4B. Plot total emissions by status (existing and future) including RoR hydroelectric

**NOTE** The map contains two "dam" layers - reservoirs, including multipurpose, hydroelectric and irrigation and a layer with RoR hydropower that is plotted with triangles

In [ ]:
colors_map <- colorspace::diverging_hcl(7, palette = "Tropic", rev = F) # OK: Cork

mya_gg <- ggplot(lims_method = "geometry_bbox")
mya_gg +
    geom_sf(fill = "lightgrey", alpha=0.9) +
    geom_sf(data=mya_neighbours_crop, fill="lightgrey", col="black", alpha=0.5) +
    geom_sf(data=mya_outline, fill="antiquewhite1", col="black", alpha=0.8, size = 5.5) +
    geom_sf_label(data=mya_neighbours_crop, aes(label = COUNTRY), fill = "white") + 
    geom_sf(data=mya_rivers, col="lightblue3") +
    geom_sf(data = res_centroids_joined,
          pch = 21,
          aes(size = r_area_km2, fill = tot_em_range),
          col = "grey31") +
    guides(fill = guide_legend(override.aes = list(size = 3))) +
    scale_fill_manual(
        values = colors_map, 
        guide = guide_legend(ncol = 1),
        name=TeX(r"(\overset{\normalsize{Total emission}}{\overset{\normalsize{ktCO$_{2e}$/yr}}})")) +
    new_scale_fill() + 
    geom_sf(
        data=ror_with_emissions_gis, pch = 24, color="grey31", #fill="white", 
        aes(fill=tot_em_range),
        show_guide = FALSE,
        alpha=0.8, size=2, lwd=0) +
    scale_fill_manual(
        values = colors_map) + 
    scale_size(breaks = c(10, 50, 100, 200, 300, 500),
        range = c(1, 15),
        guide = guide_legend(
          direction = "vertical",
          ncol = 1)) +
    labs(
        #title = "GHG emissions per construction status",
        sub = "FutureDAMS Project",
        #caption = "T. Janus, 2023",
        size = TeX(r"(\overset{\normalsize{Reservoir Area}}{\overset{\normalsize{km$^2$}}})")) +
    xlab(NULL) + ylab(NULL) +
    theme_minimal() +
    theme(
        text = element_text("Avenir Next Condensed", size=text_fontsize),
        legend.text = element_text(size = legend_text_fontsize),
        legend.title = element_text(size=legend_title_fontsize),
        legend.key.size = unit(1.3,"line"),
        axis.text.x = element_text(size = axis_label_fontsize),
        axis.text.y = element_text(size = axis_label_fontsize),
        strip.text = element_text(face="bold", hjust = 0, size=strip_fontsize),
        plot.title = element_text(hjust = 0, size=plot_title_fontsize),
        plot.caption = element_text(face = "italic"),
        panel.grid.major = element_line(
            colour = gray(0.5), linetype = "dashed", size = 0.5), 
        panel.background = element_rect(fill = "aliceblue"), 
        panel.border = element_rect(fill = NA),
        legend.direction = "vertical", legend.position = "right",
        panel.spacing = unit(1, "lines"),
        panel.ontop = FALSE
    ) +
    annotate(geom = "text", x = 820000, y = 1500000, label = "Andaman\nSea", 
     fontface = "italic", color = "grey22", size = 4) +
    annotate(geom = "text", x = 505000, y = 1950000, label = "Bay\nof\nBengal", 
     fontface = "italic", color = "grey22", size = 4) +
    ggspatial::annotation_scale(
        location = "tl",
        bar_cols = c("grey60", "white"),
        text_family = "ArcherPro Book") +
    ggspatial::annotation_north_arrow(
        location = "tl", which_north = "true",
        pad_x = unit(0.12, "in"), pad_y = unit(0.3, "in"),
        style = ggspatial::north_arrow_fancy_orienteering(
          fill = c("grey40", "white"),
          line_col = "grey20",
          text_family = "ArcherPro Book"))+
    facet_wrap(~Status) +
    xlim(bbox["xmin"]+Dx, bbox["xmax"]-Dx) + ylim(bbox["ymin"]+Dy, bbox["ymax"]-Dy) +
    coord_sf(crs = mya_epsg_code)
ggsave(
    file.path("figures","maps","total_emissions_per_status.png"), width = 27, height = 20, units = "cm")
ggsave(
    file.path("figures","maps","total_emissions_per_status.svg"), width = 27, height = 20, units = "cm")
dev.off()

In [ ]:
head(res_centroids_joined %>% select(reservoir_type, name, Status), 3)

**Emission intensity of HP :**
* tot_emisssion $E_t$ [ktCO$_{2e}$/yr]
* Flow, $Q$ in Mm$^3$/d
* Head $H$ in metres

Emission intensity is in tonnesCO$_{2e}$/GWh

P = $\eta \rho Q \, g \, H$ 

Emission intensity $\propto E_t / P$ = $ (E_{CO_2} + E_{CH_4}) / (Q \, H)$

## 4C. Print statistics about emissions of existing and future hydroelectric, irrigation and multipurpose reservoirs whilst taking into account partitioning of emissions into agricultural and hydroelectric use in multipurpose reservoirs via the split of water directed towards the two water uses.

In [ ]:
emissions_multi_by_utility <- res_centroids_joined %>%
  group_by(Status, reservoir_type) %>%
  summarise(
      number = n(),
      total_emission = sum(tot_em_int, na.rm = TRUE),
      total_area = sum(r_area_km2, na.rm = TRUE),
      total_volume = sum(r_volume_m, na.rm = TRUE) / 1e9,
      mean_depth = mean(r_mean_dep, na.rm = TRUE)
      ) %>%
    select(!geometry)
# Emissions of multipurpose reservoirs by utility
# Legend:
# emissions_total in ktCO2e/yr
# total_area in km2
# total_volume in km3
# mean_depth in m
print("Emissions of multipurpose reservoirs by utility")
head(emissions_multi_by_utility)

## 4D. Emissions per generated hydroelectric power of future and existing hydroelectric dams

###  1. Combine RoR Multipurpose and HP data with name/id/area/volume/mean_depth/generation/em_intensity
### 2. Visualise emission intensities on maps using intensity as colour and generation as size

In [ ]:
# Perform spatial left join with an sf dataframe containing multipurpose reservoirs with
# hp_fraction that quantified the percentage of 'usage' of the reservoir to generate
# hydroelectric energy. The rest of the water in the reservoir is used for agriculture
# since the multipurpose reservoirs in Myanmar are HP/irrigation only. We discount emission
# contributions to HP generation by the amount of water from the reservoir used for
# agriculture
# Set all multipurpose reservoirs to storage type
total_emissions_multi_merged$hp_type_reem = "sto"

ror_emissions_trimmed <- ror_with_emissions_gis %>% select(
    c('name','status_2_ifc','ro_r_or_sto_ifc','hp_type_reem','res_area',
       'mean','pctile_2','pctile_3','pctile_5','pctile_10','ann_gen',
       'emission_tot_int','geometry','DAM_NAME', 'Status')
) %>% rename(tot_em_int_sc=emission_tot_int) %>%
mutate(reservoir_type = "hydroelectric")


joined_data_with_hp_fraction <- joined_data %>% st_join(
    total_emissions_multi_merged %>% select(name, hp_fraction),
    by=c("name", "name")) %>%
    select(-ends_with(".y")) %>%
    rename(name = name.x)
# Scale total emissions for reservoirs in which hp_fraction is not <NA>
joined_data_with_hp_fraction <- joined_data_with_hp_fraction %>%
    mutate(
        co2_net_sc = ifelse(
            !is.na(hp_fraction), co2_net * hp_fraction, co2_net)
    ) %>%
    mutate(
        ch4_net_sc = ifelse(
            !is.na(hp_fraction), ch4_net * hp_fraction, ch4_net)
    ) %>%
    mutate(
        tot_em_sc = ifelse(
            !is.na(hp_fraction), tot_em * hp_fraction, tot_em)
    ) %>%
    mutate(
        tot_em_int_sc = ifelse(
            !is.na(hp_fraction), tot_em_int * hp_fraction, tot_em_int))

# Include RoR in joined data
joined_data_hp_multi_ror <- 
    bind_rows(joined_data_with_hp_fraction, ror_emissions_trimmed) %>%
    select(-c(
        'r_volume_m','r_maximum_','r_mean_dep','r_msocs_kg',
        'r_mghr_all','r_mghr_may','r_mghr_nov','r_mean_ann','c_mar_mm',
        'c_area_km2','n_populati','c_mean_slo','c_map_mm','c_mpet_mm',
        'c_masm_mm','c_biome','c_soil_typ','c_mean_ols','ms_length',
        'co2_net', 'ch4_net', 'n2o_mean', 'tot_em', 'tot_em_int',
        'ifc_id', 'id', 'DAM_NAME', 'em_range','tot_em_range')) %>%
    mutate(
        reservoir_type = case_when(
          is.na(reservoir_type) ~ "hydroelectric",
          TRUE ~ reservoir_type)) %>%
    mutate(
        hp_fraction = case_when(
          is.na(hp_fraction) ~ 1.0,
          TRUE ~ hp_fraction)) %>%
    mutate(Status = case_when(
        is.na(Status) & status_2_ifc == "E" ~ "Existing Hydroelectric",
        is.na(Status) ~ "Future Hydroelectric",
        TRUE ~ Status
      )) %>%
    rename(
        co2_net = co2_net_sc,
        ch4_net = ch4_net_sc,
        tot_em_net = tot_em_sc,
        tot_em = tot_em_int_sc) %>% # Back-calculate emission intensities in tCO2e/GWh
    mutate(em_intensity = tot_em / ann_gen * 1000) %>% # Re-evaluate em_intensity_range
    # Fill res_area from r_area_km2 if r_area_km2 is not NA but res_area is NA
    mutate(res_area = ifelse(!is.na(r_area_km2) & is.na(res_area), r_area_km2, res_area))

# Correct some errors in data
#joined_data_hp_multi_ror[joined_data_hp_multi_ror$name=="Thaphanseik", 'hp_type_reem'] <- "sto"

In [ ]:
head(joined_data_hp_multi_ror,2)

`joined_data_hp_multi_ror` contains storage and RoR hydroelectric unit data including emissions and emission intensities

In [ ]:
head(joined_data_hp_multi_ror,2)

In [ ]:
colnames(joined_data_hp_multi_ror)

**Descriptions of variables:**
* `co2_net`, `ch4_net` and `tot_em_net` are net per area emissions in gCO$_{2e}$/m$^2$/yr for hydroelectric and multipurpose reservoirs, i.e. rows with non-zero (non-NA) res_area.
* `ann_gen` is annual hydroelectric power generation in GWh/yr.
* `mean` is the average daily hydropower production obtained from the water resources model implemented in Pywr. It is given in MW.
* `tot_em_int` (or `tot_em` depending on the dataframe) is total emission in ktCO$_{2e}$/yr and is calculated from net (per-area) emission for reservoirs and from `ann_gen` for RoR hydropower by applying a constant emission intensity of a given value. Check `em_intensity` column for values. `em_intensity` is given in tCO$_{2e}$/GWh.

**Conversions:**
- `ann_gen` (GWh/yr)            <- `mean` (MW) * 24 * 365.25 / 1000
- `tot_em` (ktCO$_{2e}$/yr)     <- `em_intensity` (tCO$_{2e}$/GWh) * `ann_gen` (GWh/yr) / 1000

In [ ]:
joined_data_hp_multi_ror %>% group_by(Status, hp_type_reem) %>%
  summarise(
      number = n(),
      total_generation = sum(ann_gen, na.rm = TRUE),
      total_emission = sum(tot_em, na.rm = TRUE),
      )

## 2G. Plot emission intensities of hydroelectric reservoirs in Myanmar including Hydroelectric Dams, Multipurpose Dams and RoR hydropower

In [ ]:
# List multipurpose reservoirs
joined_data_hp_multi_ror %>% 
    select(name, reservoir_type, hp_type_reem) %>% 
    filter(reservoir_type=="multipurpose")

In [ ]:
joined_data_hp_multi_ror <- joined_data_hp_multi_ror %>% 
    mutate(
        #tot_em_int = ifelse(hp_type_reem=="ror" & is.na(tot_em_int), em_intensity, tot_em_int),
        tot_em = ifelse(hp_type_reem=="ror" & is.na(tot_em), em_intensity * ann_gen / 1000, tot_em)
    )

#filter(hp_type_reem=="ror") %>%  
#    mutate(tot_em = ann_gen * em_intensity / 1000) %>%
#    mutate(tot_em_int = em_intensity)

In [ ]:
joined_data_hp_multi_ror

In [ ]:
# Bin emissions intensities of hydroelectric plants in tonnesCO2eq/GWh
em_intensity_ranges <- c(
    "< 5",
    "5 - 20", 
    "20 - 50", 
    "50 - 150", 
    "150 - 300",
    "300 - 450",
    "450 - 1500",
    "> 1500"
)

joined_data_hp_multi_ror <- 
    joined_data_hp_multi_ror %>%
    mutate(
        em_intensity_range = factor(case_when(
            `em_intensity` < 5 ~ em_intensity_ranges[1],
            between(`em_intensity`, 5, 20) ~ em_intensity_ranges[2],
            between(`em_intensity`, 20, 50) ~ em_intensity_ranges[3],
            between(`em_intensity`, 50, 150) ~ em_intensity_ranges[4],
            between(`em_intensity`, 150, 300) ~ em_intensity_ranges[5],
            between(`em_intensity`, 300, 450) ~ em_intensity_ranges[6],
            between(`em_intensity`, 450, 1500) ~ em_intensity_ranges[7],
            `em_intensity` >= 1500 ~ em_intensity_ranges[8]
        ), levels = em_intensity_ranges)
    )

unique(joined_data_hp_multi_ror$Status)
hist(joined_data_hp_multi_ror$em_intensity, breaks = 100)

In [ ]:
hist(joined_data_hp_multi_ror$ann_gen, breaks = 100)

In [ ]:
write_xlsx(
    joined_data_hp_multi_ror,
    file.path("intermediate", "hp_multi_ror_emissions_and_generation.xlsx"))

In [ ]:
colors_map <- colorspace::diverging_hcl(8, palette = "Tropic", rev = F, power = 0.5) # OK: Cork
joined_data_hp_multi_ror_plot <- joined_data_hp_multi_ror %>%
     mutate(Status = replace(Status, Status == "Existing Reservoirs", "Existing Hydroelectric"))

xy_coords <- st_coordinates(joined_data_hp_multi_ror_plot$geometry)

mya_gg <- ggplot(lims_method = "geometry_bbox")
mya_gg +
    geom_sf(fill = "lightgrey", alpha=0.9) +
    geom_sf(data=mya_neighbours_crop, fill="lightgrey", col="black", alpha=0.5) +
    geom_sf(data=mya_outline, fill="antiquewhite1", col="black", alpha=0.8, size = 5.5) +
    geom_sf_label(data=mya_neighbours_crop, aes(label = COUNTRY), fill = "white") + 
    geom_sf(data=mya_rivers, col="lightblue3") +
    geom_sf(
        data = joined_data_hp_multi_ror_plot %>% 
            filter(hp_type_reem != 'ror') %>%
            arrange(desc(ann_gen)),
          pch = 21,
          aes(
              size = ann_gen, 
              fill = em_intensity_range,
          ),
          col = "grey31", alpha=0.8) +

    ggrepel::geom_text_repel(
        data = joined_data_hp_multi_ror_plot,
        aes(label = name, x=xy_coords[,'X'], y=xy_coords[,'Y']), 
        size = 2, max.overlaps = 30, color = 'black',
        segment.size=0.3) +

    geom_sf(
         data = joined_data_hp_multi_ror_plot %>% 
            filter(hp_type_reem == 'ror') %>%
            arrange(desc(ann_gen)),
          pch = 24,
          aes(
              size = ann_gen, 
              fill = em_intensity_range,
          ),
          legend=FALSE,
          col = "grey31", alpha=0.8) +
    guides(fill = guide_legend(override.aes = list(size = 3))) +
    scale_fill_manual(
        values = colors_map, 
        name=TeX(r"(\overset{\normalsize{Emission intensity}}{\overset{\normalsize{gCO$_{2e}$/kWh}}})"),
        #name="Emission footprint\ntonne CO2eq/GWh",
        guide = guide_legend(
          direction = "horizontal",
          ncol = 2)) +
    scale_size(breaks = c(100, 500, 2000, 7000, 10000, 15000, 35000),
        range = c(1, 12),
        guide = guide_legend(
          direction = "horizontal",
          ncol = 3)) +
    scale_shape_manual(
        name=TeX(r"(\overset{\normalsize{HP Type}}{\overset{\normalsize{Sto/RoR}}})"),
        values = c("sto" = 21, "ror" = 24)) +
    labs(
        #title = expression(paste("GHG emissions per generated energy in dams over 5 M", m^3)),
        sub = "FutureDAMS Project",
        fill="",
        #caption = "T. Janus, 2023",
        size = TeX(r"(\overset{\normalsize{Annual Generation}}{\overset{\normalsize{GWh/yr}}})")) +
    #guides(
    #    fill = guide_legend(
    #        override.aes = list(size = c(2.5,2.5), pch = 21)),
    #    size = guide_legend(
    #        override.aes = list(pch = 21))) +
    xlab(NULL) + ylab(NULL) +
    theme_minimal() +
    theme(       
        text = element_text("Avenir Next Condensed", size=text_fontsize),
        legend.text = element_text(size = legend_text_fontsize),
        legend.title = element_text(size=legend_title_fontsize),
        axis.text.x = element_text(size = axis_label_fontsize),
        axis.text.y = element_text(size = axis_label_fontsize),
        strip.text = element_text(face="bold", hjust = 0, size=strip_fontsize),
        plot.title = element_text(hjust = 0, size=plot_title_fontsize),
        plot.caption = element_text(face = "italic"),
        panel.grid.major = element_line(
            colour = gray(0.5), linetype = "dashed", size = 0.5), 
        panel.background = element_rect(fill = "aliceblue"), 
        panel.border = element_rect(fill = NA),
        legend.direction = "vertical", legend.position = "right",
        panel.spacing = unit(1, "lines"),
        panel.ontop = FALSE
    ) +
    annotate(geom = "text", x = 820000, y = 1500000, label = "Andaman\nSea", 
     fontface = "italic", color = "grey22", size = 4) +
    annotate(geom = "text", x = 505000, y = 1950000, label = "Bay\nof\nBengal", 
     fontface = "italic", color = "grey22", size = 4) +
    ggspatial::annotation_scale(
        location = "tl",
        bar_cols = c("grey60", "white"),
        text_family = "ArcherPro Book") +
    ggspatial::annotation_north_arrow(
        location = "tl", which_north = "true",
        pad_x = unit(0.12, "in"), pad_y = unit(0.3, "in"),
        style = ggspatial::north_arrow_fancy_orienteering(
          fill = c("grey40", "white"),
          line_col = "grey20",
          text_family = "ArcherPro Book"))+
    facet_wrap(~Status) +
    xlim(bbox["xmin"]+Dx, bbox["xmax"]-Dx) + ylim(bbox["ymin"]+Dy, bbox["ymax"]-Dy) +
    coord_sf(crs = mya_epsg_code)
ggsave(
    file.path("figures","maps","total_emissions_per_GWh.png"), width = 27, height = 20, units = "cm")
ggsave(
    file.path("figures","maps","total_emissions_per_GWh.svg"), width = 27, height = 20, units = "cm")
dev.off()

In [ ]:
# Same but from plot data
joined_data_hp_multi_ror_plot %>%
  filter(hp_type_reem == "ror") %>%
  group_by(Status, hp_type_reem) %>%
  summarise(
      number = n(),
      total_emission = sum(tot_em, na.rm = TRUE)
      )

## 3. CREATE A STATISTICS PLOT TO COMPLEMENT THE EMISSION INTENSITY PLOT OF HYDROELECTRIC RESERVOIRS

It would be useful to :
* **A.** check the relationship between per-area emissions and per HP emissions
* **B.** Plot histograms of per GWh emissions of Existing/Future HP Units. Overlay a line at emission intensity of RoR

In [ ]:
len <- length(joined_data_hp_multi_ror_plot$name)
shapes <- rep_len(x = c(21, 22, 24), length.out = len)
font_size_axis_xy <- 15
font_size_labels_xy <- 17
text_fontsize_xy <- 14
tick_size <- 15
library(scico)

In [ ]:
data_plot_stat$hp_type_reem

In [ ]:
data_plot_stat <- joined_data_hp_multi_ror_plot %>% 
    filter(hp_type_reem == 'sto' | name == "Upper Paunglaung" | name == "Lower Paunglaung") %>% 
    arrange(desc(tot_em))

data_plot_stat_ror <- joined_data_hp_multi_ror_plot %>% filter(hp_type_reem == 'ror') %>% arrange(desc(tot_em))
df_ror <-data.frame(
    tot_em_net=0,em_intensity=3, tot_em = 400, res_area = 100,
    reservoir_type = "ror")

# Select reservoirs to annotate and inspect
# Points with extraordinarily large emission intensities
points_large_em_int <- data_plot_stat %>% filter(em_intensity > 1000)

points_large_net_emissions <- data_plot_stat %>% filter(tot_em_net>3000)

points_interm_area_low_net_emissions <- data_plot_stat %>% filter(
    tot_em_net < 800 & res_area > 200 & 
    res_area < 700 & em_intensity > 10
)

points_small_tot_em_small_area_interm_net_emissions <- data_plot_stat %>%
    filter(
        tot_em_net > 1700 & tot_em_net < 2000 & em_intensity < 1000 &
        em_intensity > 10
    )

label_df <- bind_rows(
    points_large_em_int, 
    points_large_net_emissions, 
    points_interm_area_low_net_emissions,
    points_small_tot_em_small_area_interm_net_emissions
)

# Plot points
base <- ggplot(
    data = data_plot_stat, 
    aes(
        tot_em_net, em_intensity, 
        size = tot_em, 
        fill=res_area, 
        shape=factor(reservoir_type),
        color=factor(reservoir_type)
        #order=-as.numeric(factor(joined_data_hp_multi_ror$hp_type_reem))
    ), alpha=0.4
) +
geom_point(stroke=0.8) + 
# Add point representing emission intensity of RoR
geom_point(data = df_ror, size = 8, legend=FALSE, shape=24, color="#2471a3", fill="white") +

#ggforce::geom_mark_ellipse(aes(fill = res_area, filter = size > 1000)) +
# Set labels
xlab(TeX("Net aerial emission, gCO$_{2e}$/m$^2$/yr")) +
ylab(TeX("Emission intensity, gCO$_{2e}$/kWh")) +
# Add text labels to the selected points
ggrepel::geom_text_repel(
    seed=40,
    data = label_df,
    box.padding = 1.6,
    aes(label = name), size = 4, max.overlaps = 30, color = 'grey35', fill="white") +
# Set scales
scale_colour_manual(values = c("#d8b365", "#5ab4ac", "#2471a3"), name="Reservoir Type") +
scale_size(
    range = c(0, 15),
    TeX(r"(\overset{Total Emission}{\overset{\normalsize{ktCO$_{2e}$/yr}}})")) +
scale_shape_manual(values = shapes, name="Reservoir Type") +
scale_fill_gradient(
    low = "white", high = "black",
    limits=c(0,1000),
    name=TeX(r"(\overset{Reservoir Area}{\overset{\normalsize{km$^2$}}})"))+
scale_y_log10(limits = c(1,10000)) +  

# Set the theme
theme_bw() +
theme(
  text = element_text(size=text_fontsize_xy),
  #panel.border = element_rect(fill=NA, colour = "black", size=0.1),
  panel.border = element_blank(),
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
  strip.text.x = element_blank(),
  strip.background = element_rect(colour="white", fill="white"),
  axis.line=element_line(size=0.7),
  axis.text = element_text(size = font_size_axis_xy),
  axis.ticks=element_line(size=0.7),
  axis.ticks.length=unit(.12, "cm"),
  axis.title=element_text(size=font_size_labels_xy),
    legend.direction = "vertical", legend.position = "right"
) + 
guides(
    shape = guide_legend(
        override.aes = list(size = 5, width=12)),
    linetype = guide_legend(override.aes = list(size = 5)),
    color = guide_legend(
        override.aes = list(size=5, width=12)),
    size = guide_legend(
        override.aes = list(color="gray88"))
)

xbox <- cowplot::axis_canvas(base, axis = "x", coord_flip = TRUE) + 
  geom_boxplot(
      data = data_plot_stat,
      aes(
          y = tot_em_net, 
          x = factor(reservoir_type), 
          color = factor(reservoir_type))) + 
    scale_x_discrete() +
    scale_y_continuous(limits = c(0,4000)) +
    scale_colour_manual(values = c("#d8b365", "#5ab4ac", "#2471a3"), name="Reservoir Type") + 
    coord_flip()
ybox <- cowplot::axis_canvas(base, axis = "y") + 
  geom_boxplot(
      data = data_plot_stat, 
      aes(
          y = em_intensity, 
          x = factor(reservoir_type), 
          color = factor(reservoir_type))) +
    scale_y_log10(limits = c(1,10000)) +
    scale_x_discrete() +
    scale_colour_manual(values = c("#d8b365", "#5ab4ac", "#2471a3"), name="Reservoir Type")
# Add box plots visualising distributions across x and y axis
p1 <- cowplot::insert_xaxis_grob(base, xbox, grid::unit(0.7, "in"), position = "top")
p2 <- cowplot::insert_yaxis_grob(p1, ybox, grid::unit(0.7, "in"), position = "right")
cowplot::ggdraw(p2)
cowplot::save_plot('figures/maps/stats.png', p2, base_height = 6, ncol = 1, base_asp = 1.6)
cowplot::save_plot('figures/maps/stats.svg', p2, base_height = 6, ncol = 1, base_asp = 1.6)